In [1]:
import math
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from tqdm.notebook import tqdm
import random

import numba as nb
# from numba import jit, vectorize, float64

In [2]:
with open('./data/tsp_33810_1', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')
nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    # points.append(Point(float(parts[0]), float(parts[1])))
    points.append((float(parts[0]), float(parts[1])))
points = np.array(points)

In [3]:
@nb.jit(nopython=True)
def length_distance(single_point, all_points):
    return np.sqrt((all_points[:, 0]-single_point[0])**2 + (all_points[:, 1]-single_point[1])**2)


@nb.jit(nopython=True)
def calculate_travel(graph):
    n = len(graph)
    total_distance = 0.0

    for i in range(n - 1):
        total_distance += DISTANCE_MATRIX[graph[i], graph[i + 1]]

    # Add the distance to return to the starting point
    total_distance += DISTANCE_MATRIX[graph[-1], graph[0]]

    return total_distance



In [4]:
def init_path(node_count):
    ## Keep picking the next nearest neighbor to get the path.
    
    with tqdm(total=nodeCount) as pbar:
        
        pick_next = 0
        exploring_path = np.array([], dtype=int)
        exploring_path = np.append(exploring_path, pick_next)
        
        pbar.update(1)
   
        while exploring_path.size < nodeCount:
            
            neighbor_idx = np.argsort(DISTANCE_MATRIX[pick_next])
            
            mask = np.isin(neighbor_idx, exploring_path)
            neighbor_idx = neighbor_idx[~mask]
        
            pick_next = neighbor_idx[0]
            
            exploring_path = np.append(exploring_path, pick_next)

            pbar.update(1)

    total_distance = calculate_travel(exploring_path)

    return exploring_path, total_distance

In [5]:
DISTANCE_MATRIX = np.zeros((nodeCount, nodeCount))
for i in tqdm(range(len(points))):
    DISTANCE_MATRIX[i] = length_distance(points[i], points)

  0%|          | 0/33810 [00:00<?, ?it/s]

In [6]:
class Penalty:
    def __init__(self, node_count):
        self.penalty = [[0] * i for i in range(1, node_count + 1)]

    def __getitem__(self, key):
        j, i = sorted(key)
        return self.penalty[i][j]

    def __setitem__(self, key, value):
        j, i = sorted(key)
        self.penalty[i][j] = value

In [7]:
def do_2opt(pointidx1, pointidx2, path, new_distance, augmented_distance):
    if pointidx2 > pointidx1:
        slice_start = pointidx1
        slice_end = pointidx2
    else:
        slice_start = pointidx2
        slice_end = pointidx1

    new_path = np.hstack((path[:slice_start], path[slice_start:slice_end][::-1], path[slice_end:]))

    point1 = path[pointidx1-1]
    point2 = path[pointidx1]
    point3 = path[pointidx2-1]
    point4 = path[pointidx2]

    d12 = DISTANCE_MATRIX[point1][point2]
    d34 = DISTANCE_MATRIX[point3][point4]
    d13 = DISTANCE_MATRIX[point1][point3]
    d24 = DISTANCE_MATRIX[point2][point4]
    
    p12 = PENALTY[point1, point2]
    p34 = PENALTY[point3, point4]
    p13 = PENALTY[point1, point3]
    p24 = PENALTY[point2, point4]

    net_distance = d13 + d24 - d12 - d34
    augmented_distance_delta =  net_distance + lambda_factor * (p13 + p24 - p12 - p34)

    new_distance += net_distance
    augmented_distance += augmented_distance_delta 
    
    return new_path, new_distance, augmented_distance


# def pick_next_edge(pointidx1, current_path):
def pick_next_edge(point2, path):
    
    pointidx1 = np.where(path==point2)[0][0]
    point1 = path[pointidx1-1]
    point2_next = current_path[pointidx1+1] if pointidx1+1 < nodeCount else current_path[0]

    d12 = DISTANCE_MATRIX[point1][point2]
    p12 = PENALTY[point1, point2]
    
    # Repeatedly do 2-opt for all the neighbors
    next_probable_edge = {}
    max_distance_delta = float('inf')
    
    node = path[pointidx1]
    sorted_neighbor_points = np.argsort(DISTANCE_MATRIX[node])[1:SEARCH_AREA]
    
    for neighbor_idx in sorted_neighbor_points:
       
        pointidx2 = np.where(current_path==neighbor_idx)[0][0]
    
        point3 = current_path[pointidx2-1]
        point4 = current_path[pointidx2]
    
        if point4 in (point1, point2, point2_next):
            continue
    
        d12 = DISTANCE_MATRIX[point1][point2]
        d34 = DISTANCE_MATRIX[point3][point4]
        d13 = DISTANCE_MATRIX[point1][point3]
        d24 = DISTANCE_MATRIX[point2][point4]
        
        p12 = PENALTY[point1, point2]
        p34 = PENALTY[point3, point4]
        p13 = PENALTY[point1, point3]
        p24 = PENALTY[point2, point4]
    
        augmented_distance_delta = (d13 + d24 - d12 - d34) + lambda_factor * (p13 + p24 - p12 - p34)
        next_probable_edge[pointidx2] = augmented_distance_delta

    # filter candidates with min value
    if not next_probable_edge.values():
        pointidx2 = -1
        point3 = -1
        point4 = -1
        return (pointidx1, pointidx2), (point1, point2, point3, point4)
        
    min_distance_delta = min(next_probable_edge.values())
    viable_candidates = [key for key, value in next_probable_edge.items() if value == min_distance_delta]
    

    # Pick random viable candidate
    if min_distance_delta < 0:
        
        pointidx2 = random.choice(viable_candidates)
        point3 = path[pointidx2-1]
        point4 = path[pointidx2]
    else:
        # not using None as index could be 0. if None and if 0 has same effect.
        pointidx2 = -1
        point3 = -1
        point4 = -1
    
    return (pointidx1, pointidx2), (point1, point2, point3, point4)


def add_penalty(path):
    util_nodes = {}
    
    for idx, node in enumerate(path):
        node_out = path[idx+1] if idx+1 < nodeCount else path[0]
        dist = DISTANCE_MATRIX[node][node_out]
        penalty = (1 + PENALTY[node, node_out])
        util = dist / (1 + penalty)
        util_nodes[(node, node_out)] = util

    max_util = max(util_nodes.values())
    viable_candidates = [(node, node_out) for (node, node_out), value in util_nodes.items() if value == max_util]

    for node, node_out in viable_candidates:
        PENALTY[node, node_out] += 1
        ACTIVATE[node] = 1
        ACTIVATE[node_out] = 1

In [8]:
step_limit = 100000
PENALTY = Penalty(nodeCount)
ACTIVATE = np.ones(nodeCount, dtype=int)

current_path, current_distance = init_path(nodeCount)



  0%|          | 0/33810 [00:00<?, ?it/s]

In [9]:
# current_path = init_path()np.array([0, 33, 5, 2, 28, 45, 9, 10, 3, 46, 24, 34, 23, 12, 36, 6, 26, 47, 27, 41, 8, 4, 35, 13, 7, 19, 40, 18, 11, 42, 29, 43, 21, 37, 20, 25, 1, 22, 31, 39, 50, 38, 15, 44, 16, 14, 30, 48, 32, 17, 49])
# random.shuffle(current_path)
# current_distance = calculate_travel(current_path)
current_augmented_distance = current_distance

optimal_distance = current_distance
optimal_path = deepcopy(current_path)

In [10]:
alpha = 0.1
SEARCH_AREA = nodeCount//10
lambda_factor = alpha * current_distance/nodeCount
lambda_factor, SEARCH_AREA, current_distance

(244.5402021337388, 3381, 82679042.34141709)

In [ ]:
retry = 0
last_distance = float('inf')

for _ in tqdm(range(step_limit)):
    while sum(ACTIVATE) > 0:
        for explore_node in range(ACTIVATE.size):
            
            if not ACTIVATE[explore_node]:
                continue
        
            indexes, points_visited = pick_next_edge(explore_node, current_path)
            
            pt_idx1, pt_idx2 = indexes
            p1, p2, p3, p4 = points_visited
            
            if pt_idx2 == -1:
                ACTIVATE[explore_node] = 0
                # print('Setting zero: ', explore_node)
                continue
            
            current_path, current_distance, current_augmented_distance = do_2opt(pt_idx1, pt_idx2, current_path, current_distance, current_augmented_distance)
            # print(current_distance, current_augmented_distance)
            # print(current_path)

            ACTIVATE[p1] = 1
            ACTIVATE[p2] = 1
            ACTIVATE[p3] = 1
            ACTIVATE[p4] = 1
            
            if current_distance < optimal_distance:
                optimal_path = deepcopy(current_path)
                optimal_distance = current_distance 

    
    
    add_penalty(current_path)

    print(current_distance, current_augmented_distance, optimal_distance)
    
    if _ % 100 == 0:
        if optimal_distance < last_distance:
            last_distance = optimal_distance
            retry = 0
        else:
            retry += 1

    if retry > 20:
        break
    
        

  0%|          | 0/100000 [00:00<?, ?it/s]

72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72443367.83116378 72443367.83116378 72443367.83116378
72440011.08324869 72439766.54304655 72440011.08324869
72440011.08324869 72439766.54304655 72440011.08324869
72440049.32259552 72439560.24219126 72440011.08324869
72440216.29305916 72439482.67245276 72440011.08324869
72440082.69278465 72439104.53197612 72440011.08324869
72440011.08324869 72439032.92244016 72440011.08324869
72440011.08324869 72439032.92244016 72440011.08324869
72440011.08324869 72439032.92244016 72440011.08324869
72440011.08324869 72439032.9

In [ ]:
print(current_distance, current_augmented_distance, optimal_distance)

In [ ]:
output_data = '%.2f' % optimal_distance + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, optimal_path))

with open(r'./results/tsp_33810_1.txt', 'w') as file:
    # Write the Python code to the file
    file.write(output_data)